# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG, display, Markdown
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


# Load training dataset

In [2]:
train_dataframe = pd.read_csv('dataset/1. istanbul/train_data.csv')

# Visualize some training data

In [3]:
train_dataframe.head()

,Subject ID,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,Class information
0,1,1.488,0.000090,0.900,0.794,2.699,8.334,0.779,4.517,4.609,...,187.576,160,159,0.006065,0.000416,0.000,0,0.000,23,1
1,1,0.728,0.000038,0.353,0.376,1.059,5.864,0.642,2.058,3.180,...,234.505,170,169,0.005181,0.000403,2.247,0,0.000,23,1
2,1,1.220,0.000074,0.732,0.670,2.196,8.719,0.875,4.347,5.166,...,211.442,1431,1427,0.006071,0.000474,10.656,1,0.178,23,1
3,1,2.502,0.000123,1.156,1.634,3.469,13.513,1.273,5.263,8.771,...,220.230,94,92,0.004910,0.000320,0.000,0,0.000,23,1
4,1,3.509,0.000167,1.715,1.539,5.145,9.112,1.040,3.102,4.927,...,225.162,117,114,0.004757,0.000380,18.182,1,13.318,23,1


# Prepare training data

In [4]:
train_x = train_dataframe.drop(['Subject ID', 'Class information', 'UPDRS'], axis=1)
train_x = train_x.as_matrix()
print(train_x.shape)

(1040, 26)


# Prepare training labels

In [5]:
train_y = train_dataframe['Class information']
train_y = train_y.as_matrix()
print(train_y.shape)
train_y = np_utils.to_categorical(train_y)
print(train_y.shape)

(1040,)
(1040, 2)


# Prepare test data

In [6]:
test_dataframe = pd.read_csv('dataset/1. istanbul/test_data.csv')
test_x = test_dataframe.drop(['Subject ID', 'Class information'], axis=1)
test_x = test_x.as_matrix()
print(test_x.shape)

(168, 26)


# Prepare test labels

In [7]:
test_y = test_dataframe['Class information']
test_y = test_y.as_matrix()
print(test_y.shape)
test_y = np_utils.to_categorical(test_y)
print(test_y.shape)

(168,)
(168, 2)


# Combine train and test set

In [8]:
x = np.vstack([train_x, test_x])
y = np.vstack([train_y, test_y])
print(x.shape)
print(y.shape)

(1208, 26)
(1208, 2)


# Normalize data

In [9]:
mins = np.min(x, axis=0)
maxs = np.max(x, axis=0)
rng = maxs - mins
x = 1.0 - (((1.0 - 0.0) * (maxs - x)) / rng)
pd.DataFrame(x).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.096023,0.110551,0.106928,0.054303,0.106923,0.186514,0.266893,0.162585,0.056981,0.142044,...,0.034343,0.193353,0.204351,0.107383,0.106783,0.378729,0.062423,0.000000,0.000000,0.000000
1,0.042716,0.042501,0.038278,0.023294,0.038318,0.125260,0.215176,0.065922,0.037237,0.150903,...,0.049169,0.238354,0.295334,0.114094,0.113499,0.295624,0.060411,0.025488,0.000000,0.000000
2,0.077225,0.089595,0.085843,0.045104,0.085882,0.196062,0.303133,0.155902,0.064677,0.158903,...,0.043064,0.204329,0.250621,0.960403,0.958361,0.379296,0.071557,0.120874,0.083333,0.002575
3,0.167146,0.152809,0.139056,0.116617,0.139134,0.314949,0.453379,0.191910,0.114487,0.367528,...,0.035815,0.298746,0.267658,0.063087,0.061786,0.270109,0.047233,0.000000,0.000000,0.000000
4,0.237778,0.209959,0.209212,0.109570,0.209245,0.205808,0.365421,0.106962,0.061375,0.278121,...,0.038016,0.299027,0.277220,0.078523,0.076561,0.255738,0.056711,0.206243,0.083333,0.192688


# Shuffle data

In [10]:
x, y = shuffle(x, y)
pd.DataFrame(y).head()

,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,1.0,0.0


# Tuning parameters

In [11]:
epochs = 15000
folds = 5
batch_size = 20
dropout = 0.5

# Folder paths to store outputs.
root_directory = 'results/results-crossvalidation-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"

# Create model amd save its params in storage

In [12]:
def create_model(date, param, index, folds):
    input_x = Input(shape=(x.shape[1],))

    output_y = Dense(units = param['layer_1'], activation='relu')(input_x)
    output_y = Dropout(dropout)(output_y)

    output_y = Dense(units = param['layer_2'], activation='relu')(output_y)
    output_y = Dropout(dropout)(output_y)
    
    output_y = Dense(units = param['layer_3'], activation='relu')(output_y)
    output_y = Dropout(dropout)(output_y)
    
    if (param['layer_4'] > 0):
        output_y = Dense(units = param['layer_4'], activation='relu')(output_y)
        output_y = Dropout(dropout)(output_y)

    output_y = Dense(units = y.shape[1], activation='softmax')(output_y)
    model = Model(input_x, output_y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    file = open(directory + "params.txt", "a")
    file.write('Iteration: %d out of folds %d' % (index+1, folds))
    file.write("optimizer: %s, layer 1: %d, layer 2: %d, layer 3: %d, layer 4: %d" % (param['optimizer_text'], param['layer_1'], param['layer_2'], param['layer_3'], param['layer_4']))
    file.close()
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=param['optimizer'], loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [13]:
def get_callbacks(date):
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
    csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
    model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
    callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
    return callbacks

# Fit the model using stratified k fold cross-validation

In [14]:
stratifiedKFold = StratifiedKFold(n_splits=folds, shuffle=True)

y_temp = []
for index in range(y.shape[0]):
    label = y[index]
    y_temp.append(0 if label[0] == 0 and label[1] == 1 else 1)

index = 1
for train_indices, val_indices in stratifiedKFold.split(x, y_temp):
    x_train, x_val = x[train_indices], x[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]
    date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    param = {
        'optimizer': keras.optimizers.Adam(lr=0.0001, decay=0.0001), 
        'optimizer_text': 'Adam, lr: 0.0001, decay: 0.0001', 
        'layer_1': 4, 'layer_2': 8, 'layer_3': 16, 'layer_4': 0
    }
    callbacks = get_callbacks(date)
    model = None
    model = create_model(date, param, index, folds)
    print('Starting to train iteration #' + str(index))
    history = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), shuffle=False, batch_size=batch_size, epochs=epochs, callbacks=callbacks)
    print('Finished training train iteration #%d, accuracy %f, loss %f, val accuracy %f, val loss %f' % (index, history.history['acc'][-1], history.history['loss'][-1], history.history['val_acc'][-1], history.history['val_loss'][-1]))
    index = index + 1

Starting to train iteration #1
Train on 966 samples, validate on 242 samples
Epoch 1/15000
966/966 [==============================] - 0s - loss: 0.6922 - acc: 0.5642 - val_loss: 0.6927 - val_acc: 0.5289
Epoch 2/15000
966/966 [==============================] - 0s - loss: 0.6913 - acc: 0.5590 - val_loss: 0.6919 - val_acc: 0.5289
Epoch 3/15000
966/966 [==============================] - 0s - loss: 0.6922 - acc: 0.5704 - val_loss: 0.6912 - val_acc: 0.5620
Epoch 4/15000
966/966 [==============================] - 0s - loss: 0.6917 - acc: 0.5559 - val_loss: 0.6905 - val_acc: 0.5661
Epoch 5/15000
966/966 [==============================] - 0s - loss: 0.6902 - acc: 0.5652 - val_loss: 0.6897 - val_acc: 0.5661
Epoch 6/15000
966/966 [==============================] - 0s - loss: 0.6890 - acc: 0.5600 - val_loss: 0.6890 - val_acc: 0.5785
Epoch 7/15000
966/966 [==============================] - 0s - loss: 0.6920 - acc: 0.5714 - val_loss: 0.6884 - val_acc: 0.5744
Epoch 8/15000
966/966 [==================

966/966 [==============================] - 0s - loss: 0.6761 - acc: 0.5839 - val_loss: 0.6798 - val_acc: 0.5868
Epoch 65/15000
966/966 [==============================] - 0s - loss: 0.6801 - acc: 0.5890 - val_loss: 0.6797 - val_acc: 0.5868
Epoch 66/15000
966/966 [==============================] - 0s - loss: 0.6815 - acc: 0.5828 - val_loss: 0.6795 - val_acc: 0.5909
Epoch 67/15000
966/966 [==============================] - 0s - loss: 0.6807 - acc: 0.5725 - val_loss: 0.6796 - val_acc: 0.5868
Epoch 68/15000
966/966 [==============================] - 0s - loss: 0.6809 - acc: 0.5870 - val_loss: 0.6795 - val_acc: 0.5868
Epoch 69/15000
966/966 [==============================] - 0s - loss: 0.6821 - acc: 0.5797 - val_loss: 0.6794 - val_acc: 0.5868
Epoch 70/15000
966/966 [==============================] - 0s - loss: 0.6778 - acc: 0.5797 - val_loss: 0.6793 - val_acc: 0.5868
Epoch 71/15000
966/966 [==============================] - 0s - loss: 0.6799 - acc: 0.5807 - val_loss: 0.6792 - val_acc: 0.5909

966/966 [==============================] - 0s - loss: 0.6766 - acc: 0.5859 - val_loss: 0.6756 - val_acc: 0.5785
Epoch 129/15000
966/966 [==============================] - 0s - loss: 0.6781 - acc: 0.5828 - val_loss: 0.6755 - val_acc: 0.5785
Epoch 130/15000
966/966 [==============================] - 0s - loss: 0.6786 - acc: 0.5901 - val_loss: 0.6755 - val_acc: 0.5785
Epoch 131/15000
966/966 [==============================] - 0s - loss: 0.6757 - acc: 0.5859 - val_loss: 0.6754 - val_acc: 0.5785
Epoch 132/15000
966/966 [==============================] - 0s - loss: 0.6808 - acc: 0.5807 - val_loss: 0.6754 - val_acc: 0.5785
Epoch 133/15000
966/966 [==============================] - 0s - loss: 0.6774 - acc: 0.5787 - val_loss: 0.6754 - val_acc: 0.5785
Epoch 134/15000
966/966 [==============================] - 0s - loss: 0.6789 - acc: 0.5828 - val_loss: 0.6754 - val_acc: 0.5785
Epoch 135/15000
966/966 [==============================] - 0s - loss: 0.6777 - acc: 0.5818 - val_loss: 0.6753 - val_acc:

966/966 [==============================] - 0s - loss: 0.6756 - acc: 0.5973 - val_loss: 0.6735 - val_acc: 0.5785
Epoch 193/15000
966/966 [==============================] - 0s - loss: 0.6712 - acc: 0.5932 - val_loss: 0.6735 - val_acc: 0.5785
Epoch 194/15000
966/966 [==============================] - 0s - loss: 0.6735 - acc: 0.5973 - val_loss: 0.6735 - val_acc: 0.5785
Epoch 195/15000
966/966 [==============================] - 0s - loss: 0.6776 - acc: 0.5797 - val_loss: 0.6735 - val_acc: 0.5785
Epoch 196/15000
966/966 [==============================] - 0s - loss: 0.6715 - acc: 0.5932 - val_loss: 0.6734 - val_acc: 0.5785
Epoch 197/15000
966/966 [==============================] - 0s - loss: 0.6721 - acc: 0.5921 - val_loss: 0.6734 - val_acc: 0.5785
Epoch 198/15000
966/966 [==============================] - 0s - loss: 0.6769 - acc: 0.5756 - val_loss: 0.6733 - val_acc: 0.5744
Epoch 199/15000
966/966 [==============================] - 0s - loss: 0.6804 - acc: 0.5880 - val_loss: 0.6734 - val_acc:

966/966 [==============================] - 0s - loss: 0.6773 - acc: 0.5890 - val_loss: 0.6716 - val_acc: 0.5744
Epoch 257/15000
966/966 [==============================] - 0s - loss: 0.6769 - acc: 0.5901 - val_loss: 0.6716 - val_acc: 0.5785
Epoch 258/15000
966/966 [==============================] - 0s - loss: 0.6705 - acc: 0.5932 - val_loss: 0.6715 - val_acc: 0.5744
Epoch 259/15000
966/966 [==============================] - 0s - loss: 0.6788 - acc: 0.5839 - val_loss: 0.6716 - val_acc: 0.5785
Epoch 260/15000
966/966 [==============================] - 0s - loss: 0.6667 - acc: 0.5932 - val_loss: 0.6716 - val_acc: 0.5785
Epoch 261/15000
966/966 [==============================] - 0s - loss: 0.6727 - acc: 0.5890 - val_loss: 0.6716 - val_acc: 0.5785
Epoch 262/15000
966/966 [==============================] - 0s - loss: 0.6842 - acc: 0.5787 - val_loss: 0.6717 - val_acc: 0.5785
Epoch 263/15000
966/966 [==============================] - 0s - loss: 0.6766 - acc: 0.5880 - val_loss: 0.6717 - val_acc:

...

967/967 [==============================] - 0s - loss: 0.6526 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14747/15000
967/967 [==============================] - 0s - loss: 0.6569 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14748/15000
967/967 [==============================] - 0s - loss: 0.6577 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14749/15000
967/967 [==============================] - 0s - loss: 0.6458 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14750/15000
967/967 [==============================] - 0s - loss: 0.6535 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14751/15000
967/967 [==============================] - 0s - loss: 0.6526 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14752/15000
967/967 [==============================] - 0s - loss: 0.6563 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14753/15000
967/967 [==============================] - 0s - loss: 0.6509 - acc: 0.5698 - val_loss: 0.6

967/967 [==============================] - 0s - loss: 0.6562 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14810/15000
967/967 [==============================] - 0s - loss: 0.6421 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14811/15000
967/967 [==============================] - 0s - loss: 0.6600 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14812/15000
967/967 [==============================] - 0s - loss: 0.6552 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14813/15000
967/967 [==============================] - 0s - loss: 0.6536 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14814/15000
967/967 [==============================] - 0s - loss: 0.6542 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14815/15000
967/967 [==============================] - 0s - loss: 0.6525 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14816/15000
967/967 [==============================] - 0s - loss: 0.6557 - acc: 0.5698 - val_loss: 0.6

967/967 [==============================] - 0s - loss: 0.6508 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14872/15000
967/967 [==============================] - 0s - loss: 0.6589 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14873/15000
967/967 [==============================] - 0s - loss: 0.6455 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14874/15000
967/967 [==============================] - 0s - loss: 0.6586 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14875/15000
967/967 [==============================] - 0s - loss: 0.6595 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14876/15000
967/967 [==============================] - 0s - loss: 0.6541 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14877/15000
967/967 [==============================] - 0s - loss: 0.6515 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14878/15000
967/967 [==============================] - 0s - loss: 0.6434 - acc: 0.5698 - val_loss: 0.6

967/967 [==============================] - 0s - loss: 0.6500 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14935/15000
967/967 [==============================] - 0s - loss: 0.6492 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14936/15000
967/967 [==============================] - 0s - loss: 0.6666 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14937/15000
967/967 [==============================] - 0s - loss: 0.6584 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14938/15000
967/967 [==============================] - 0s - loss: 0.6617 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14939/15000
967/967 [==============================] - 0s - loss: 0.6489 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14940/15000
967/967 [==============================] - 0s - loss: 0.6613 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14941/15000
967/967 [==============================] - 0s - loss: 0.6573 - acc: 0.5698 - val_loss: 0.6

967/967 [==============================] - 0s - loss: 0.6557 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14998/15000
967/967 [==============================] - 0s - loss: 0.6519 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 14999/15000
967/967 [==============================] - 0s - loss: 0.6562 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Epoch 15000/15000
967/967 [==============================] - 0s - loss: 0.6526 - acc: 0.5698 - val_loss: 0.6308 - val_acc: 0.5685
Finished training train iteration #5, accuracy 0.569804, loss 0.652637, val accuracy 0.568465, val loss 0.630791


# The End